In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.keras as tfk
import shap
from keras_visualizer import visualizer

In [ ]:
def inputshape(freq, sig):
    """
    Prepares the input data for the CNN model by combining frequency and signal arrays.
    
    Parameters:
    freq : np.array
        Frequency values (2D array)
    sig : np.array
        EMI signal values (2D array)
        
    Returns:
    x : np.array
        3D array of shape (samples, frequencies, 2) 
        where the last dimension contains [frequency, signal] for each sample.
    """
    m, n = np.shape(freq)
    x = np.zeros((m, n, 2))
    x[:, :, 0] = freq
    x[:, :, 1] = sig
    return x

def loaddata():
    """
    Loads all input and output data for the model from text files:
    - 'y.txt'      : target compressive strengths and silica fume content and molar concentration features
    - 'cond.txt'   : EMI signal data for main samples
    - 'cond_g.txt' : EMI signal data for reference/control samples
    
    Normalizes EMI signals, reshapes arrays for CNN input, and prepares
    one-hot encoded age data.
    
    Returns:
    m, n         : dimensions of EMI input
    x_mols       : silica fume content and molar concentration features
    x_emi        : normalized EMI signals (reshaped for CNN)
    x_g          : reference EMI signals
    x_age        : one-hot encoded age features
    y            : target compressive strength values
    """
    temp = np.loadtxt("y.txt")
    y = temp[:, 0]
    x_mols = temp[:, 1:]

    
    sign = np.loadtxt("cond.txt")
    sign=sign/np.max(sign)
    sign_g = np.loadtxt("cond_g.txt")
    sign_g=sign_g/np.max(sign)

    x_emi = np.transpose(sign)
    m , n = np.shape(x_emi)
    x_emi = np.reshape(x_emi, (m, n, 1))
    x_g = np.transpose(sign_g)

    m , n, c = np.shape(x_emi)
    x_age = np.identity(4)
    x_age = np.transpose(np.tile(x_age, int(m/4)))
    return m, n, x_mols, x_emi, x_g, x_age, y

    
    
def normalize(x, m ,s):
    x_norm = (x-m)/s
    return x_norm

def R2(x, y, DNN):
    """
    Computes the coefficient of determination (R²) for a trained model.
    
    Parameters:
    x   : np.array or list of np.arrays
          Input features to the model (can be multi-input for CNN)
    y   : np.array
          True target values
    DNN : tf.keras.Model
          Trained deep learning model (CNN)
          
    Returns:
    R2  : float
          Coefficient of determination, measures goodness-of-fit.
          R² = 1 indicates perfect predictions; R² = 0 indicates model predicts the mean.
    """
    m = np.mean(y)
    f = DNN.predict(x, verbose = 0)[:, 0]
    tot = y-m; sse=f-y
    ss_tot = np.sum(np.power(tot, 2))
    ss_sse = np.sum(np.power(sse, 2))
    R2 = 1-(ss_sse/ss_tot)
    return R2
    
def CNN(m, n):
    """
    Defines the multi-input CNN model for predicting compressive strength.
    This model has three inputs:
    1. EMI signal input (n frequencies, 1 channel)
    2. Age input (one-hot encoded)
    3. Silica fume content and molar concentration features input (2 features)
    
    Architecture:
    - EMI input: 5 Conv1D layers with increasing/decreasing filters, flattened, dense layer
    - Age input: Dense layers
    - Silica fume content and molar concentration input: Dense layers
    - All three branches concatenated and passed through final Dense layer for regression
    
    Parameters:
    m : int
        Number of samples (not used directly in model definition)
    n : int
        Number of frequency points in EMI input
        
    Returns:
    mod : tf.keras.Model
        Compiled multi-input CNN model
    """
    inp_cnn = tfk.layers.Input(shape=(n, 1))
    x1 = tfk.layers.Conv1D(64, kernel_size=3, activation='relu')(inp_cnn)
    x2 = x1 = tfk.layers.Conv1D(128, kernel_size=5, activation='relu')(x1)
    x3 = x1 = tfk.layers.Conv1D(256, kernel_size=7, activation='relu')(x2)
    x4 = x1 = tfk.layers.Conv1D(128, kernel_size=5, activation='relu')(x3)
    x5 = x1 = tfk.layers.Conv1D(64, kernel_size=3, activation='relu')(x4)
    x6 = tfk.layers.Flatten()(x5)
    x7 = tfk.layers.Dense(150, activation='relu')(x6)

    y1 = tfk.layers.Dense(10, activation='relu')(x7)
    
    inp_age = tfk.layers.Input(shape=(4))
    xx1 = tfk.layers.Dense(150, activation='relu')(inp_age)
    y2 = tfk.layers.Dense(10, activation='relu')(xx1)

    inp_moll = tfk.layers.Input(shape=(2))
    xxx1 = tfk.layers.Dense(150, activation='relu')(inp_moll)
    y3 = tfk.layers.Dense(10, activation='relu')(xxx1)

    merge = tfk.layers.Concatenate()([y1, y2, y3])

    out = tfk.layers.Dense(1)(merge)

    mod = tfk.models.Model([inp_cnn, inp_age, inp_moll], out)
    return mod


In [ ]:
##################### inputs and outputs ##################
m, n, x_mols, x_emi, x_g, x_age, y = loaddata()
model = CNN(m, n)

In [ ]:
iteration = 700
optimizer = tf.optimizers.Adam()  # or lr=0.001 (deprecated alias)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
hist = model.fit([x_emi, x_age, x_mols], y, batch_size= 36, epochs=iteration, validation_split=0.0)

In [ ]:
# ============================================
# PLOTTING TRAINING LOSS
# ============================================

# Extract the loss history from model training
# 'hist' is assumed to be the History object returned by model.fit()
J = hist.history['loss']

# Plot the loss curve


plt.figure(figsize=(20, 5))
plt.plot(np.arange(0, 700), J, marker='o', linestyle='-', linewidth=2)  # Plot loss vs epoch
plt.title("Loss function")      # Figure title
plt.xlabel("Epochs")            # X-axis label
plt.ylabel("Cost (J)")          # Y-axis label (cost function)
plt.grid(True)                  # Add grid for readability
plt.show()

# ============================================
# PREDICTION USING TRAINED MODEL
# ============================================

# Predict the compressive strength using the trained CNN model
# Inputs: x_emi (EMI signal), x_age (one-hot encoded age), x_mols (molecular features)
yper = model.predict([x_emi, x_age, x_mols])

# Flatten predictions to 1D array for easier handling in plots or error metrics
yper = np.ndarray.flatten(yper)

In [ ]:
# Scatter plot of predicted vs actual compressive strength
# Red '+' markers indicate individual predictions
plt.plot(y, yper, "+r", markersize=12)  

# Plot reference line y = x (perfect prediction)
# Blue line shows where predicted = actual
plt.plot([30, 60], [30, 60], "-b")  

# Add grid for readability
plt.grid(True)

# Label axes
plt.xlabel("Real Values (MPa)")       # Actual compressive strength
plt.ylabel("Predicted Values (MPa)")  # Predicted compressive strength

# Title (optional)
plt.title("CNN Predictions vs Actual Values")

# Show figure
plt.show()

In [ ]:
# ----------------------------
# 1. Define background dataset for SHAP
# ----------------------------
# Background samples are used by DeepExplainer to estimate expected model output.
# Here, we take:
# - EMI signal: first 9 samples reshaped for CNN input
# - Age: first 9 age samples
# - Molar and silica features: repeated first sample 9 times
NomberOfBasesets = 3
# change the number of base sets to 9 when using the complete data. for the sake of demonstration it's set to 3 in the demo data.
background_emi = np.reshape(x_g, (-1, 120, 1))  # reshape EMI reference data
background_age = x_age[0:NomberOfBasesets, :]                  # select first 9 age samples
background_mols = np.reshape(np.tile(x_mols[0, :], NomberOfBasesets), (-1, 2))  # repeat first molecular feature row 9 times

# ----------------------------
# 2. Initialize SHAP explainer
# ----------------------------
# DeepExplainer is used for deep learning models (CNN here).
# It calculates feature importance (SHAP values) for each input.
explainer = shap.DeepExplainer(model, [background_emi, background_age, background_mols])

# ----------------------------
# 3. Compute SHAP values for all samples
# ----------------------------
shap_values = explainer.shap_values([x_emi, x_age, x_mols])

# ----------------------------
# 4. Separate SHAP values for each input branch
# ----------------------------
shap_emi = np.squeeze(shap_values[0])   # SHAP values for EMI frequency input
shap_age = np.squeeze(shap_values[1])   # SHAP values for age input
# Note: shap_values[2] corresponds to molecular features if needed

In [ ]:
bin_size = 4
num_bins = shap_emi.shape[1] // bin_size

# 2. Average SHAP values over each bin
shap_binned = np.array([shap_emi[:, i*bin_size:(i+1)*bin_size].mean(axis=1) 
                        for i in range(num_bins)]).T  # shape: (num_samples, num_bins)

# 3. Create new feature names for 20 kHz bins
G_binned = [f"{20+i*20} to {40+i*20} KHz" for i in range(num_bins)]

# 4. Plot violin plot with binned SHAP values
shap.plots.violin(shap_binned, x_emi[:, :num_bins].squeeze(), 
                  feature_names=G_binned, max_display=num_bins, sort=False, show=False)

# 5. Save figure
plt.gcf().savefig("emi_shap_violin_20kHz.png", dpi=600, bbox_inches='tight')

In [ ]:
np.savetxt("shap_e.txt", shap_binned, delimiter = ';')

In [ ]:
R2([x_emi, x_age, x_mols], y, model)

In [ ]:
# Assign SHAP values for EMI input to a variable for saving
shap_cond = shap_emi

# Save SHAP values as a text file for reproducibility or external analysis
# - File name: 'shap_cond.txt'
# - Delimiter: semicolon
np.savetxt("shap_cond.txt", shap_cond, delimiter=';')

# --------------------------------------------
# Editor Notes / Caption:
# --------------------------------------------
# 1. Purpose: Preserve SHAP feature importance values for the EMI input in an external file.
# 2. Format: Each row corresponds to one sample; each column corresponds to one frequency point.
# 3. Usage: This file can be used for further analysis, plotting, or verification of results outside Python.